In [1]:
from lightning import Fabric
import numpy as np
from sklearn.linear_model import SGDRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange

from causalpruner.sgd_pruner import ParamDataset

torch.set_float32_matmul_precision('medium')

In [2]:
fb = Fabric()
fb.launch()

In [31]:
# ds = ParamDataset("./checkpoints/resnet50_imagenet_causalpruner_1_10_1_0_0.9/weights/0",
#                   "./checkpoints/resnet50_imagenet_causalpruner_1_10_1_0_0.9/loss/0",
#                   False)
ds = ParamDataset("./checkpoints/lenet_cifar10_causalpruner_10_10_10_0_0.9/weights/0",
                  "./checkpoints/lenet_cifar10_causalpruner_10_10_10_0_0.9/loss/0",
                  False)
# dl = DataLoader(ds, batch_size=16)
dl = DataLoader(ds, batch_size=16)
dl = fb.setup_dataloaders(dl)

In [ ]:
X, Y = next(iter(dl))

In [172]:
torch.topk(torch.mean(X, dim=0), k=1, largest=True)

torch.return_types.topk(
values=tensor([1.2987], device='cuda:0'),
indices=tensor([651459], device='cuda:0'))

In [173]:
X[:,651459]

tensor([8.8818e-16, 0.0000e+00, 2.4754e-02,  ..., 2.2111e-01, 4.8597e+00,
        6.0523e+00], device='cuda:0')

In [174]:
Y

tensor([-0.0017, -0.0014, -0.0012,  ..., -0.0014, -0.0013, -0.0066],
       device='cuda:0')

In [121]:
X_mean = ds.weights_zstats.mean.to('cuda')
X_std = ds.loss_zstats.mean.to('cuda')
Y_mean = ds.loss_zstats.mean.to('cuda')
Y_std = ds.loss_zstats.std.to('cuda')

In [118]:
global_max_X = torch.max(X)
global_min_X = torch.min(X)
global_max_Y = torch.max(Y)
global_min_Y = torch.min(Y)
print(global_max_X, global_min_X, global_max_Y, global_min_Y)

tensor(7.1146, device='cuda:0') tensor(0., device='cuda:0') tensor(-0.0008, device='cuda:0') tensor(-0.0031, device='cuda:0')


In [116]:
X_new = (X - global_min_X) / (global_max_X - global_min_X)
Y_new = (Y - global_min_Y) / (global_max_Y - global_min_Y)

In [122]:
X_new_new = (X_new - X_mean) / X_std
Y_new_new = (Y_new - Y_mean) / Y_std

In [123]:
X_new_new

tensor([[2.6247e-08, 2.2915e-08, 2.3065e-08,  ..., 3.3308e-12, 4.0490e-12,
         2.8315e-14],
        [2.9165e-08, 2.8779e-08, 2.9874e-08,  ..., 3.3308e-12, 4.0490e-12,
         2.8315e-14],
        [2.0695e-08, 2.0638e-08, 2.7549e-08,  ..., 3.3795e-12, 4.0490e-12,
         2.8315e-14],
        ...,
        [2.2785e-08, 1.9386e-08, 2.0441e-08,  ..., 2.6482e-12, 4.0490e-12,
         2.8315e-14],
        [3.0109e-08, 3.0659e-08, 3.2408e-08,  ..., 3.3795e-12, 4.0490e-12,
         2.8315e-14],
        [2.4386e-08, 2.2558e-08, 2.5918e-08,  ..., 3.3795e-12, 3.6590e-12,
         2.8315e-14]], device='cuda:0')

In [111]:
a = torch.abs(ds.weights_zstats.mean)
a[a==0] = 1e10
torch.topk(a, k=10, largest=False)

torch.return_types.topk(
values=tensor([1.0804e-24, 2.4309e-24, 4.3216e-24, 4.3216e-24, 4.3216e-24, 1.7286e-23,
        1.7286e-23, 1.7286e-23, 1.7286e-23, 1.7286e-23]),
indices=tensor([266675, 372074, 585360, 367703,  98680,  16859, 292007, 285480, 367821,
        267978]))

In [112]:
index = 372074
print(ds.weights_zstats.mean[index])
print(ds.weights_zstats.std[index])
print(X[:,372074])

tensor(2.4309e-24)
tensor(3.7434e-23)
tensor([-8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05,  1.5629e-02, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05, -8.0146e-05,
        -8.01

In [14]:
X_abs = torch.abs(X)
Y_abs = torch.abs(Y)

print(f"{torch.max(X_abs)}; {torch.min(X_abs)}")
print(f"{torch.max(Y_abs)}; {torch.min(Y_abs)}")

8.500299453735352; 0.0
5.054473876953125e-05; 7.152557373046875e-07


In [9]:
X = X.cpu().numpy()
Y = np.ravel(Y.cpu().numpy())

In [10]:
num_params = ds.weights_zstats.num_params

In [11]:
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
Y_mean = np.mean(Y)
Y_std = np.std(Y)

In [12]:
X_new = (X - X_mean) / (X_std * np.sqrt(num_params))
Y_new = (Y - Y_mean) / Y_std

/tmp/ipykernel_134005/1931566632.py:1: RuntimeWarning: divide by zero encountered in divide
  X_new = (X - X_mean) / (X_std * np.sqrt(num_params))
/tmp/ipykernel_134005/1931566632.py:1: RuntimeWarning: invalid value encountered in divide
  X_new = (X - X_mean) / (X_std * np.sqrt(num_params))


In [13]:
X_new

array([[-2.17508277e-05, -1.28595575e-05,  2.30965685e-04, ...,
        -2.54521478e-06, -2.11545514e-04, -1.78197137e-04],
       [-3.65201326e-04, -3.20843508e-04, -3.17644415e-04, ...,
        -1.68832659e-04, -2.11545514e-04, -1.78197137e-04],
       [-3.30951414e-04, -2.92304467e-04, -3.17644415e-04, ...,
        -1.68832659e-04, -2.11545514e-04, -1.78197137e-04],
       ...,
       [ 1.23407913e-03,  7.78623566e-04,  1.86744306e-04, ...,
        -1.68832659e-04, -2.11545514e-04, -1.78197137e-04],
       [-3.61395767e-04, -3.20130028e-04, -3.16193618e-04, ...,
        -1.68832659e-04, -2.11545514e-04, -1.78197137e-04],
       [-3.61395767e-04, -3.21081345e-04, -3.17232247e-04, ...,
        -1.68832659e-04, -2.11545514e-04, -1.78197137e-04]], dtype=float32)

In [14]:
trainer = SGDRegressor(loss="squared_error", penalty="l1", alpha=1e-3, fit_intercept=False, shuffle=True, verbose=True)

In [ ]:
trainer.fit(X_new, Y)
print(trainer.n_iter_)

In [95]:
trainer.coef_

array([0., 0., 0., ..., 0., 0., 0.])

In [5]:
dl = DataLoader(ds, batch_size=16, shuffle=True)
dl = fb.setup_dataloaders(dl)

In [7]:
num_params = ds.weights_zstats.num_params
l1_regularization_coeff = 0
lr = 1e-3

In [7]:
model = nn.Linear(num_params, 1, bias=False)
nn.init.zeros_(model.weight)
abs_weight = torch.abs(model.weight)
print(f"{torch.max(abs_weight)}; {torch.min(abs_weight)}")
optimizer = optim.SGD(model.parameters(), lr=lr)
model, optimizer = fb.setup(model, optimizer)

0.0; 0.0


In [4]:
ds.weights_zstats

ZStats(num_params=25502912, mean=tensor([5.1708e-11, 4.7973e-11, 4.6359e-11,  ..., 4.5812e-14, 1.2386e-13,
        3.3172e-13]), std=tensor([7.7607e-11, 7.2559e-11, 7.0380e-11,  ..., 4.2518e-13, 1.2495e-12,
        2.0406e-12]))

In [5]:
ds.loss_zstats

ZStats(num_params=1, mean=tensor(-0.0108), std=tensor(0.0014))

In [8]:
num_epochs = int(np.ceil(np.log(num_params / len(ds))))
print(num_epochs)

9


In [10]:
model = nn.Linear(num_params, 1, bias=False)
nn.init.zeros_(model.weight)
abs_weight = torch.abs(model.weight)
print(f"{torch.max(abs_weight)}; {torch.min(abs_weight)}")
optimizer = optim.SGD(model.parameters(), lr=lr)
model, optimizer = fb.setup(model, optimizer)
model.train()
dl = DataLoader(ds, batch_size=16, shuffle=True)
dl = fb.setup_dataloaders(dl)
dl_iter = iter(dl)

0.0; 0.0


In [ ]:
model.train()
for epoch in trange(num_epochs):
    total_loss = 0
    num_batches = 0
    for idx, (X, Y) in enumerate(tqdm(dl)):
        optimizer.zero_grad(set_to_none=True)
        outputs = model(X)
        Y = Y.view(outputs.size())
        loss = F.mse_loss(outputs, Y, reduction="mean")
        total_loss += loss.item()
        num_batches += 1
        tqdm.write(f"epoch: {epoch + 1}; batch: {idx + 1}; Loss: {loss.item()}")
        fb.backward(loss)
        optimizer.step()
    avg_loss = total_loss / num_batches
    tqdm.write(f"epoch: {epoch + 1}; avg_loss: {avg_loss}")

In [12]:
torch.max(torch.abs(model.weight))

tensor(1.6422e-13, device='cuda:0', grad_fn=<MaxBackward1>)

In [13]:
torch.min(torch.abs(model.weight))

tensor(0., device='cuda:0', grad_fn=<MinBackward1>)

In [15]:
last_layer_num_params = 2048000
last_layer_start = num_params - last_layer_num_params
model.weight[0, last_layer_start:]

tensor([-1.2169e-17, -5.2730e-17, -6.9182e-17,  ..., -1.5129e-17,
        -5.6211e-17, -8.1764e-17], device='cuda:0', grad_fn=<SliceBackward0>)

In [16]:
model.weight[0, :100]

tensor([-2.0681e-14, -1.9123e-14, -1.8859e-14, -1.9659e-14, -2.0971e-14,
        -2.0334e-14, -2.1363e-14, -1.8792e-14, -1.7077e-14, -1.6719e-14,
        -1.6679e-14, -1.7886e-14, -1.8264e-14, -1.9038e-14, -1.8170e-14,
        -1.5651e-14, -1.4909e-14, -1.5020e-14, -1.5414e-14, -1.6501e-14,
        -1.6782e-14, -1.8724e-14, -1.6246e-14, -1.4884e-14, -1.4541e-14,
        -1.4242e-14, -1.5773e-14, -1.5926e-14, -1.9396e-14, -1.7806e-14,
        -1.5509e-14, -1.4270e-14, -1.4282e-14, -1.4743e-14, -1.5674e-14,
        -1.9984e-14, -1.7581e-14, -1.5993e-14, -1.5042e-14, -1.5233e-14,
        -1.5711e-14, -1.6868e-14, -2.0246e-14, -1.8906e-14, -1.7864e-14,
        -1.7230e-14, -1.6191e-14, -1.6810e-14, -1.7744e-14, -1.3945e-14,
        -1.2679e-14, -1.2212e-14, -1.2214e-14, -1.3270e-14, -1.3417e-14,
        -1.4047e-14, -1.1975e-14, -1.0136e-14, -9.6365e-15, -9.3358e-15,
        -1.1114e-14, -1.1149e-14, -1.1714e-14, -1.1557e-14, -8.9648e-15,
        -8.5017e-15, -8.1683e-15, -9.0271e-15, -9.6

In [32]:
X, Y = next(iter(dl))

In [33]:
torch.mean(X[:, :100])

tensor(5.5467e-10, device='cuda:0')

In [34]:
torch.mean(X[:, -10000:-1000])

tensor(0.0078, device='cuda:0')